In [1]:
from nsga2.estimator import NSGAIIRegressor
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv('../data/lexicase_paper/d_concrete.txt', sep=',')

# DEAP interface requires X and y to be numpy arrays, not pandas dataframes
X = df.drop('label', axis=1).values
y = df['label'].values

X_train, X_test, y_train, y_test = train_test_split(X, y)

estimator = NSGAIIRegressor(**{
    'pop_size'        : 40, 
    'max_gen'         : 100,
    'max_depth'       : 7,  # 8
    'max_size'        : 2**7, # 75
    'objectives'      : ['error', 'size'],
    'initialization'  : 'uniform',
    'pick_criteria'   : 'error', # error, MCDM
    'validation_size' : 0.33,
    'simplify'        : True,
    
    # Either you use smart variation (just 1 cx and 1 mutation)
    'smart_variation' : True,

    # Or you use mabs (4 mutations)
    'use_mab'         : False,
    'use_context'     : False,

    'simplification_method' : 'bottom_up',
    'simplification_tolerance' : 1e-2,
    'verbosity'       : 1,
    'survival'       : 'tournament'
}).fit(X_train, y_train)

gen	evals	best_size	n_simplifications	n_new_hashes	avg train error	avg train size	avg val error	avg val size	med train error	med train size	med val error	med val size	std train error	std train size	std val error	std val size	min train error	min train size	min val error	min val size	max train error	max train size	max val error	max val size
0  	40   	1        	76               	312         	               	              	             	            	               	              	             	            	               	              	             	            	               	              	             	            	               	              	             	            
1  	40   	12       	3                	19          	               	              	             	            	               	              	             	            	               	              	             	            	               	              	             	            	               	              	    

In [2]:
# Should not give erros even without mabs
pd.DataFrame(estimator.variator.mab.pull_history).iloc[:30]

,t,arm,reward,update,delta_error,gen
0,0,lsh_mutate,0.0,0,"[-0.0, -0.0]",1
1,1,lsh_mutate,0.0,0,"[-inf, -0.0]",1
2,2,cx,0.0,0,"[7780.095876208898, -0.0]",1
3,3,lsh_mutate,0.0,0,"[-846.3013442600732, -4.0]",1
4,4,cx,1.0,0,"[8087380330.652251, 5.0]",1
5,5,lsh_mutate,0.0,0,"[-0.0, -0.0]",1
6,6,lsh_mutate,0.0,0,"[-739.3754062632597, -4.0]",1
7,7,lsh_mutate,0.0,0,"[-97.21703288781305, -3.0]",1
8,8,lsh_mutate,0.0,0,"[-0.0, -0.0]",1
9,9,lsh_mutate,0.0,0,"[-0.0, -0.0]",1


In [3]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse

model      = str(estimator.best_estimator_).replace("ARG", "x_")
size       = len(estimator.best_estimator_)
complexity = size
depth      = estimator.best_estimator_.height

print(model)
print(size)
print(complexity)
print(depth)

for metric, fn, (data_X, data_y) in [
    ('train_r2',  r2_score, (X_train, y_train)),
    ('test_r2',   r2_score, (X_test,  y_test )),
    ('train_mse', mse,      (X_train, y_train)),
    ('test_mse',  mse,      (X_test,  y_test )),
]:
    score = np.nan
    try:
        score = fn(estimator.predict(data_X), data_y)
        print(f"{metric} : {score}")
    except ValueError:
        print(f"(Failed to calculate {metric}")

for ind in estimator.archive_:
    print(ind.fitness, ind)

exp(expm1(arctan(x_7)))
4
4
3
train_r2 : -0.8866824683756263
test_r2 : -0.9529856080508619
train_mse : 201.88168100533426
test_mse : 197.60114479564203
(209.12808426391544, 4.0) exp(expm1(arctan(ARG7)))


In [4]:
if False:
    print( len(list(estimator.simplifier.pop_hash.keys())) )

    n_keys =  len(list(estimator.simplifier.pop_hash.keys()))

    for key in list(estimator.simplifier.pop_hash.keys())[:n_keys]:
        print(key)
        for ind in estimator.simplifier.pop_hash[key]:
            print(" -", ind)

In [5]:
display(df.describe())

A = np.maximum(df['x0'], df['x4']+29.657).values
B = df['x6'].values

print(np.std(A) * (A - np.mean(A))[:5])
print(np.std(B) * (B - np.mean(B))[:5])

,x0,x1,x2,x3,x4,x5,x6,x7,label
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


[27036.47147454 27036.47147454  5361.93013146  5361.93013146
 -8624.67751114]
[ -7819.81227097  -7819.81227097 -14391.05039658 -14391.05039658
   4160.67675074]
